### Define Periods of Study

First pass
- Select a usual week day am peak period
- Find day with good data quality coverage

Second pass
- AM vs PM peak
- Weekday vs Weekend
- Summer vs Winter
- Bank Holiday 


Dates of interest in first pass
- March 15th 2022
    - 7am - 9am
    - 10am - 2pm
    - 4pm - 6pm


### From documentation:
https://sumo.dlr.de/docs/Demand/Routes_from_Observation_Points.html#computing_flows


Computing Flows

The next step is to use the computed routes and flow amounts from the real-world detectors to compute flows across the modelled network. The flows are given to dfrouter using --measure-files <STRING> (or -f <DETECTOR_FLOWS>[,<DETECTOR_FLOWS>]+. They are assumed to be stored in CSV-format using ';' as dividing character. The file should look as follows:

```
Detector;Time;qPKW;qLKW;vPKW;vLKW
myDet1;0;10;2;100;80
... further entries ...
```

This means the first time has to name the entries (columns). Their order is not of importance, but at least the following columns must be included:

    Detector: A string holding the id of the detector this line describes; should be one of the ids used in <DETECTOR_FILE>
    Time: The time period begin that this entry describes (in minutes)
    qPKW: The number of passenger cars that drove over the detector within this time period
    vPKW: The average speed of passenger cars that drove over the detector within this time period in km/h


In [ ]:
#Process:

# For each sensor included
# Get time series data for time period
# Split flow between number of lanes
# Output in desired format

In [14]:
import geopandas as gpd
from helper_functions import traversal_time, haversine_distance, test_stationarity, get_ts, analyse_time_series_of_sensor
import mysql.connector
import pandas as pd

#Set up cursor object to query time-series database
host = "localhost"
user = "chris"
password = "password"
database = "midas"

# Establish a database connection
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor(buffered=True)

In [24]:
start_date = '2022-01-01'
end_date = '2022-12-31 23:45:00'

filter_start_date = '2022-05-11 07:00'
filter_end_date = '2022-05-11 09:00'
filter_datetime_index = pd.date_range(start=filter_start_date, end=filter_end_date, freq='15T')

/tmp/ipykernel_149303/4168392915.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  filter_datetime_index = pd.date_range(start=filter_start_date, end=filter_end_date, freq='15T')


In [16]:
sensors = gpd.read_file("Data/Networks/route1/dfrouter/sensor_snapping_new_network.json")

In [ ]:
# Construct Time Series for DFROUTER

In [33]:
ts_append_list = []

for next_id in list(sensors['Sensor ID']):

    ts, ts_all = get_ts(next_id,start_date,end_date,cursor)

    #Filter TS on date studied
    filter_ts = ts_all.loc[filter_datetime_index]
    filter_ts['time'] = range(0, len(filter_ts) * 15, 15)

    # Get sensor details
    sensor_details = sensors[sensors['Sensor ID'] == next_id]

    for i,r in filter_ts.iterrows():
        veh_per_lane = int(r['All Lanes_interpolated'] / sensor_details['Num lanes on new network'].values[0])
        for lane in range(sensor_details['Num lanes on new network'].values[0]):
            ts_append = {}
            ts_append['Detector'] = '{}_{}'.format(sensor_details['Sensor ID'].values[0],lane)
            ts_append['Time'] = r['time']
            ts_append['qPKW'] = veh_per_lane
            ts_append['vPKW'] = sensor_details['Lane speed'].values[0]
            ts_append_list.append(ts_append)

/home/chris/projects/Adaptive_Traffic_Simulator/helper_functions.py:103: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  datetime_index = pd.date_range(start=start_date, end=end_date, freq='15T')
/home/chris/projects/Adaptive_Traffic_Simulator/helper_functions.py:121: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ts_all[variable_name].bfill(inplace=True)
/home/chris/projects/Adaptive_Traffic_Simulator/helper_functions.py:121: FutureWarning: A value is trying to be set on a 

In [36]:
ts_for_dfroute = pd.DataFrame(ts_append_list)
ts_for_dfroute.to_csv('Data/Networks/route1/dfrouter/detector_ts.csv', sep=';', index=False)

In [35]:
ts_for_dfroute

,Detector,Time,qPKW,vPKW
0,5889/1_0,0,125,31.29
1,5889/1_1,0,125,31.29
2,5889/1_0,15,114,31.29
3,5889/1_1,15,114,31.29
4,5889/1_0,30,103,31.29
...,...,...,...,...
481,M20/6404A_1,105,39,31.29
482,M20/6404A_2,105,39,31.29
483,M20/6404A_0,120,81,31.29
484,M20/6404A_1,120,81,31.29
